In [1]:
import numpy as np
import pandas as pd
import random as rn
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC # "Support Vector Classifier" 
from sklearn.linear_model import LogisticRegression
#from rotation_forest import RotationTreeClassifier
#from rotation_forest import RotationForestClassifier

In [2]:
df = pd.read_csv("C:\\Users\\abhic\\Modelling_in_Operation_Management\\3-Healthcare\\data.csv")

In [3]:
df = df.drop('Unnamed: 32', axis=1)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
print(df.shape)

(569, 32)


In [5]:
y = df['diagnosis']
X = df.drop(['id','diagnosis'], axis=1)

In [6]:
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))[0]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:
# PCA
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler() 
X_trainp = sc.fit_transform(X_train) 
X_testp = sc.transform(X_test) 
from sklearn.decomposition import PCA 
pca = PCA(n_components = 7) 
X_trainp = pca.fit_transform(X_train) 
X_testp= pca.transform(X_test) 

In [12]:
f1 = []
acc = []
n = 50
print("Naive Bayes:")
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    ypred = nb.predict(X_test)
    f1.append(f1_score(y_test, ypred, average='weighted'))
    acc.append(accuracy_score(y_test, ypred))
print("Avg F1-Score",np.mean(f1))
print("Max F1-Score",np.max(f1))
print("Min F1-Score",np.min(f1))
print("Avg Accuracy",np.mean(acc))
print("Max Accuracy",np.max(acc))
print("Min Accuracy",np.min(acc))

Naive Bayes:
Avg F1-Score 0.9405770627302146
Max F1-Score 0.9737857247406666
Min F1-Score 0.9013593586615545
Avg Accuracy 0.9410526315789474
Max Accuracy 0.9736842105263158
Min Accuracy 0.9035087719298246


In [8]:
nb = GaussianNB()
# without pca
nb.fit(X_train, y_train)
ypred = nb.predict(X_test)
print("Without pca",f1_score(y_test, ypred, average='weighted'))
nb.fit(X_trainp, y_train)
ypredp = nb.predict(X_testp)
print("With pca",f1_score(y_test, ypredp, average='weighted'))

Without pca 0.9470870627956459
With pca 0.8996627671698025


In [9]:
kmeans = KMeans(n_clusters = 2, random_state = 0).fit(pca.fit_transform(X))
print("With",f1_score(y, kmeans.labels_, average='weighted'))
kmeans = KMeans(n_clusters = 2, random_state = 0).fit(X)
print("Without",f1_score(y, kmeans.labels_, average='weighted'))

With 0.8443374522699525
Without 0.8443374522699525


In [10]:
clust = AgglomerativeClustering(n_clusters = 2).fit(X)
f1_score(y, clust.labels_, average='weighted')

0.7483522251447848

In [13]:
f1 = []
acc = []
n = 50
print("KNN:")
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    neigh = KNeighborsClassifier(n_neighbors = 5)
    # without pca
    neigh.fit(X_train, y_train)
    ypred = neigh.predict(X_test)
    f1.append(f1_score(y_test, ypred, average='weighted'))
    acc.append(accuracy_score(y_test, ypred))
print("Avg F1-Score",np.mean(f1))
print("Max F1-Score",np.max(f1))
print("Min F1-Score",np.min(f1))
print("Avg Accuracy",np.mean(acc))
print("Max Accuracy",np.max(acc))
print("Min Accuracy",np.min(acc))

KNN:
Avg F1-Score 0.9314165655401772
Max F1-Score 0.9736401936741494
Min F1-Score 0.873910469457975
Avg Accuracy 0.9319298245614036
Max Accuracy 0.9736842105263158
Min Accuracy 0.8771929824561403


In [11]:
neigh = KNeighborsClassifier(n_neighbors = 5)
# without pca
neigh.fit(X_train, y_train)
ypred = neigh.predict(X_test)
print("Without pca",f1_score(y_test, ypred, average='weighted'))
# with pca
neigh.fit(X_trainp, y_train)
ypredp = neigh.predict(X_testp)
print("With pca",f1_score(y_test, ypredp, average='weighted'))

Without pca 0.9380859556298152
With pca 0.9380859556298152


In [14]:
f1 = []
acc = []
n = 50
print("MLP:")
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    clf = MLPClassifier(solver='sgd', alpha=0.00001, hidden_layer_sizes=(5, 2), random_state=1, max_iter=1000)
    # without pca
    clf.fit(X_train, y_train)
    ypred = clf.predict(X_test)
    f1.append(f1_score(y_test, ypred, average='weighted'))
    acc.append(accuracy_score(y_test, ypred))
print("Avg F1-Score",np.mean(f1))
print("Max F1-Score",np.max(f1))
print("Min F1-Score",np.min(f1))
print("Avg Accuracy",np.mean(acc))
print("Max Accuracy",np.max(acc))
print("Min Accuracy",np.min(acc))

MLP:


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Avg F1-Score 0.8603917903449597
Max F1-Score 0.9560099381918393
Min F1-Score 0.1822738664843928
Avg Accuracy 0.8749122807017542
Max Accuracy 0.956140350877193
Min Accuracy 0.3508771929824561


In [12]:
clf = MLPClassifier(solver='sgd', alpha=0.00001, hidden_layer_sizes=(5, 2), random_state=1, max_iter=1000)
# without pca
clf.fit(X_train, y_train)
ypred = clf.predict(X_test)
print("Without pca",f1_score(y_test, ypred, average='weighted'))
# without pca
clf.fit(X_trainp, y_train)
ypredp = clf.predict(X_testp)
print("With pca",f1_score(y_test, ypredp, average='weighted'))

Without pca 0.9290184921763869
With pca 0.8900134952766531


In [15]:
f1 = []
acc = []
n = 50
print("SVM:")
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    svm = SVC(kernel='linear') 
    # without pca
    svm.fit(X_train, y_train) 
    ypred = svm.predict(X_test)
    f1.append(f1_score(y_test, ypred, average='weighted'))
    acc.append(accuracy_score(y_test, ypred))
print("Avg F1-Score",np.mean(f1))
print("Max F1-Score",np.max(f1))
print("Min F1-Score",np.min(f1))
print("Avg Accuracy",np.mean(acc))
print("Max Accuracy",np.max(acc))
print("Min Accuracy",np.min(acc))

SVM:
Avg F1-Score 0.9556066986284167
Max F1-Score 0.9911898500306574
Min F1-Score 0.9205740079728058
Avg Accuracy 0.9557894736842104
Max Accuracy 0.9912280701754386
Min Accuracy 0.9210526315789473


In [13]:
#SVM

svm = SVC(kernel='linear') 
# without pca
svm.fit(X_train, y_train) 
ypred = svm.predict(X_test)
print("Without pca",f1_score(y_test, ypred, average='weighted'))
# with pca
svm.fit(X_trainp, y_train) 
ypredp = svm.predict(X_testp)
print("With pca",f1_score(y_test, ypredp, average='weighted'))

Without pca 0.9554882874504246
With pca 0.9467638691322903


In [22]:
f1 = []
acc = []
n = 50
print("Logistic:")
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    lr = LogisticRegression(random_state=0, solver='liblinear')
    # without pca
    lr.fit(X_train, y_train)
    ypred=lr.predict(X_test)
    f1.append(f1_score(y_test, ypred, average='weighted'))
    acc.append(accuracy_score(y_test, ypred))
print("Avg F1-Score",np.mean(f1))
print("Max F1-Score",np.max(f1))
print("Min F1-Score",np.min(f1))
print("Avg Accuracy",np.mean(acc))
print("Max Accuracy",np.max(acc))
print("Min Accuracy",np.min(acc))

Logistic:
Avg F1-Score 0.948414260647288
Max F1-Score 0.9824561403508771
Min F1-Score 0.910884806954094
Avg Accuracy 0.9485964912280701
Max Accuracy 0.9824561403508771
Min Accuracy 0.9122807017543859


In [14]:
# logistic regression
lr = LogisticRegression(random_state=0, solver='liblinear')
# without pca
lr.fit(X_train, y_train)
ypred=lr.predict(X_test)
print("Without pca",f1_score(y_test, ypred, average='weighted'))
# with pca
lr.fit(X_trainp, y_train)
ypredp=lr.predict(X_testp)
print("With pca",f1_score(y_test, ypredp, average='weighted'))

Without pca 0.9554882874504246
With pca 0.9467638691322903


In [20]:
f1 = []
acc = []
n = 50
print("Random Forest:")
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    rf = RandomForestClassifier(n_estimators = 100, random_state = 0) 
    # without pca
    rf.fit(X_train, y_train)
    ypred=rf.predict(X_test)
    f1.append(f1_score(y_test, ypred, average='weighted'))
    acc.append(accuracy_score(y_test, ypred))
print("Avg F1-Score",np.mean(f1))
print("Max F1-Score",np.max(f1))
print("Min F1-Score",np.min(f1))
print("Avg Accuracy",np.mean(acc))
print("Max Accuracy",np.max(acc))
print("Min Accuracy",np.min(acc))

Random Forest:
Avg F1-Score 0.9582495710739555
Max F1-Score 0.9825957298309063
Min F1-Score 0.9202549255687112
Avg Accuracy 0.958421052631579
Max Accuracy 0.9824561403508771
Min Accuracy 0.9210526315789473


In [15]:
# random forest
rf = RandomForestClassifier(n_estimators = 100, random_state = 0) 
# without pca
rf.fit(X_train, y_train)
ypred=rf.predict(X_test)
print("Without pca",f1_score(y_test, ypred, average='weighted'))
# with pca
rf.fit(X_trainp,y_train)
ypredp=rf.predict(X_testp)
print("Without pca",f1_score(y_test, ypredp, average='weighted'))

Without pca 0.9467638691322903
Without pca 0.9645092460881936


In [19]:
f1 = []
acc = []
n = 50
print("Decision Tree:")
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    dt = tree.DecisionTreeClassifier()
    # without pca
    dt.fit(X_train, y_train)
    ypred=dt.predict(X_test)
    f1.append(f1_score(y_test, ypred, average='weighted'))
    acc.append(accuracy_score(y_test, ypred))
print("Avg F1-Score",np.mean(f1))
print("Max F1-Score",np.max(f1))
print("Min F1-Score",np.min(f1))
print("Avg Accuracy",np.mean(acc))
print("Max Accuracy",np.max(acc))
print("Min Accuracy",np.min(acc))

Decision Tree:
Avg F1-Score 0.9261119610684616
Max F1-Score 0.9737571441832094
Min F1-Score 0.8589529379003062
Avg Accuracy 0.926140350877193
Max Accuracy 0.9736842105263158
Min Accuracy 0.8596491228070176


In [16]:
# decision tree

dt = tree.DecisionTreeClassifier()
# without pca
dt.fit(X_train, y_train)
ypred=dt.predict(X_test)
print("Without pca",f1_score(y_test, ypred, average='weighted'))
# with pca
dt.fit(X_trainp,y_train)
ypredp=dt.predict(X_testp)
print("With pca",f1_score(y_test, ypredp, average='weighted'))

Without pca 0.9470870627956459
With pca 0.8842695473711037


In [17]:
#temp = [None]*len(y_test)
#X_test.insert(loc = 30, value = temp, column = 'diagnosis')

In [18]:
df1 = pd.read_csv("C:\\Users\\abhic\\Modelling_in_Operation_Management\\3-Healthcare\\breast-cancer.data", header=None)

In [19]:
df1.head()

,0,1,2,3,4,5,6,7,8,9
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [20]:
le = LabelEncoder()
m, n = df1.shape
for i in range(n):
    df1.iloc[:,i] = le.fit_transform(df1.iloc[:,i])

In [41]:
y1 = df1[9]
X1 = df1.drop(9, axis=1)

In [42]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=1)

In [21]:
# Bayesian Network
from pomegranate import *
#model = BayesianNetwork.from_samples(X_train1, algorithm='exact')
# without pca
#model.fit(X_train1)
#ypred1=model.predict(X_test1)
#print("Without pca",f1_score(y_test1, ypred1, average='weighted'))
# with pca
#model.fit(X_trainp)
#ypredp=model.predict(X_testp)
#print("With pca",f1_score(y_test1, ypredp, average='weighted'))

In [17]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree._tree import DTYPE
from sklearn.ensemble.forest import ForestClassifier
from sklearn.utils import resample, gen_batches, check_random_state
from sklearn.decomposition import PCA


def random_feature_subsets(array, batch_size, random_state=1234):
    """ Generate K subsets of the features in X """
    random_state = check_random_state(random_state)
    features = list(range(array.shape[1]))
    random_state.shuffle(features)
    for batch in gen_batches(len(features), batch_size):
        yield features[batch]


class RotationTreeClassifier(DecisionTreeClassifier):
    def __init__(self,
                 n_features_per_subset=3,
                 rotation_algo='pca',
                 criterion="gini",
                 splitter="best",
                 max_depth=None,
                 min_samples_split=2,
                 min_samples_leaf=1,
                 min_weight_fraction_leaf=0.,
                 max_features=1.0,
                 random_state=None,
                 max_leaf_nodes=None,
                 class_weight=None,
                 presort=False):

        self.n_features_per_subset = n_features_per_subset
        self.rotation_algo = rotation_algo

        super(RotationTreeClassifier, self).__init__(
            criterion=criterion,
            splitter=splitter,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            min_weight_fraction_leaf=min_weight_fraction_leaf,
            max_features=max_features,
            max_leaf_nodes=max_leaf_nodes,
            class_weight=class_weight,
            random_state=random_state,
            presort=presort)

    def rotate(self, X):
        if not hasattr(self, 'rotation_matrix'):
            raise NotFittedError('The estimator has not been fitted')

        return np.dot(X, self.rotation_matrix)

    def pca_algorithm(self):
        """ Deterimine PCA algorithm to use. """
        if self.rotation_algo == 'randomized':
            return PCA(svd_solver='randomized', random_state=self.random_state)
        elif self.rotation_algo == 'pca':
            return PCA()
        else:
            raise ValueError("`rotation_algo` must be either "
                             "'pca' or 'randomized'.")

    def _fit_rotation_matrix(self, X):
        random_state = check_random_state(self.random_state)
        n_samples, n_features = X.shape
        self.rotation_matrix = np.zeros((n_features, n_features),
                                        dtype=np.float32)
        for i, subset in enumerate(
                random_feature_subsets(X, self.n_features_per_subset,
                                       random_state=self.random_state)):
            # take a 75% bootstrap from the rows
            x_sample = resample(X, n_samples=int(n_samples*0.75),
                                random_state=10*i)
            pca = self.pca_algorithm()
            #print(x_sample[:, subset])
            pca.fit(x_sample[:, subset])
            self.rotation_matrix[np.ix_(subset, subset)] = pca.components_

    def fit(self, X, y, sample_weight=None, check_input=True):
        self._fit_rotation_matrix(X)
        super(RotationTreeClassifier, self).fit(self.rotate(X), y,
                                                sample_weight, check_input)

    def predict_proba(self, X, check_input=True):
        return  super(RotationTreeClassifier, self).predict_proba(self.rotate(X),
                                                                  check_input)

    def predict(self, X, check_input=True):
        return super(RotationTreeClassifier, self).predict(self.rotate(X),
                                                           check_input)

    def apply(self, X, check_input=True):
        return super(RotationTreeClassifier, self).apply(self.rotate(X),
                                                         check_input)

    def decision_path(self, X, check_input=True):
        return super(RotationTreeClassifier, self).decision_path(self.rotate(X),
                                                                 check_input)

class RotationForestClassifier(ForestClassifier):
    def __init__(self,
                 n_estimators=10,
                 criterion="gini",
                 n_features_per_subset=3,
                 rotation_algo='pca',
                 max_depth=None,
                 min_samples_split=2,
                 min_samples_leaf=1,
                 min_weight_fraction_leaf=0.,
                 max_features=1.0,
                 max_leaf_nodes=None,
                 bootstrap=False,
                 oob_score=False,
                 n_jobs=1,
                 random_state=None,
                 verbose=0,
                 warm_start=False,
                 class_weight=None):
        super(RotationForestClassifier, self).__init__(
            base_estimator=RotationTreeClassifier(),
            n_estimators=n_estimators,
            estimator_params=("n_features_per_subset", "rotation_algo",
                              "criterion", "max_depth", "min_samples_split",
                              "min_samples_leaf", "min_weight_fraction_leaf",
                              "max_features", "max_leaf_nodes",
                              "random_state"),
            bootstrap=bootstrap,
            oob_score=oob_score,
            n_jobs=n_jobs,
            random_state=random_state,
            verbose=verbose,
            warm_start=warm_start,
            class_weight=class_weight)

        self.n_features_per_subset = n_features_per_subset
        self.rotation_algo = rotation_algo
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.max_features = max_features
        self.max_leaf_nodes = max_leaf_nodes

In [74]:
# rotation forest
rf = RotationTreeClassifier(n_features_per_subset=3)
# without pca
rf.fit(X_train.values, y_train)
ypred = rf.predict(X_test)
print("F1-Score",f1_score(y_test, ypred, average='weighted'))
print("Accuracy",accuracy_score(y_test, ypred))

F1-Score 0.9554882874504246
Accuracy 0.956140350877193


In [115]:
# rotation forest
rf = RotationForestClassifier(n_estimators=50, n_features_per_subset=3)
# without pca
rf.fit(X_train.values, y_train)
ypred = rf.predict(X_test)
print("F1-Score",f1_score(y_test, ypred, average='weighted'))
print("Accuracy",accuracy_score(y_test, ypred))

F1-Score 0.9467638691322903
Accuracy 0.9473684210526315


In [136]:
def fun(X, y, n, ne, nf):
    f1 = []
    acc = []
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        # rotation forest
        rf = RotationForestClassifier(n_estimators=ne, n_features_per_subset=nf)
        # without pca
        rf.fit(X_train.values, y_train)
        ypred = rf.predict(X_test)
        f1.append(f1_score(y_test, ypred, average='weighted'))
        acc.append(accuracy_score(y_test, ypred))
    print("Avg F1-Score",np.mean(f1))
    print("Max F1-Score",np.max(f1))
    print("Min F1-Score",np.min(f1))
    print("Avg Accuracy",np.mean(acc))
    print("Max Accuracy",np.max(acc))
    print("Min Accuracy",np.min(acc))

In [121]:
fun(X, y, 50, 50, 3)

F1-Score 0.9573211378391026
Accuracy 0.957017543859649


In [160]:
parameters = {'n_estimators':list(range(1,100,5)), 'n_features_per_subset':list(range(1,30))}
rf = RotationForestClassifier()
grid = GridSearchCV(rf, parameters , cv=3)
grid.fit(X_train, y_train)
x=grid.best_params_
sol=[x['n_estimators'],x['n_features_per_subset']]
print("Best Parameters:", sol)

Best Parameters: [31, 10]


In [126]:
print("Best Parameters:", sol)

Best Parameters: [86, 5]


In [137]:
fun(X, y, 50, sol[0], sol[1])
# Avg F1-Score 0.9844730924178237
# Max F1-Score 0.9912019876383678
# Min F1-Score 0.9823481781376517
# Avg Accuracy 0.9845614035087717
# Max Accuracy 0.9912280701754386
# Min Accuracy 0.9824561403508771

Avg F1-Score 0.9618092994006375
Max F1-Score 0.9912412610473553
Min F1-Score 0.920972619433514
Avg Accuracy 0.9619298245614035
Max Accuracy 0.9912280701754386
Min Accuracy 0.9210526315789473


In [138]:
fun(X, y, 50, sol[0], sol[1])

Avg F1-Score 0.9641210356531273
Max F1-Score 0.991255984633468
Min F1-Score 0.9208930903769
Avg Accuracy 0.9642105263157894
Max Accuracy 0.9912280701754386
Min Accuracy 0.9210526315789473


In [161]:
fun(X, y, 50, sol[0], sol[1])

Avg F1-Score 0.9652395711293044
Max F1-Score 0.9912103433752112
Min F1-Score 0.9207843137254901
Avg Accuracy 0.9652631578947367
Max Accuracy 0.9912280701754386
Min Accuracy 0.9210526315789473
